In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
data_dir = "/home/solomon/Downloads/mnist"
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting /home/solomon/Downloads/mnist/train-images-idx3-ubyte.gz
Extracting /home/solomon/Downloads/mnist/train-labels-idx1-ubyte.gz
Extracting /home/solomon/Downloads/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/solomon/Downloads/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SIZE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

In [4]:
def inference(input_tensor, avg_class, weight1, biases1, weight2, biases2):
    if avg_class is None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weight1) + biases1)
        return tf.matmul(layer1, weight2) + biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weight1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weight2)) + avg_class.average(biases2)

In [9]:
x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")

# 隐藏层参数
weight1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))

# 输出层参数
weight2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

# 计算前向传播结果
y = inference(x, None, weight1, biases1, weight2, biases2)

In [11]:
# 定义训练轮数
global_step = tf.Variable(0, trainable=False)

# 定义滑动平均类
variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)

# 除辅助变量外，对神经网络中的所有变量使用滑动平均
variable_averages_op = variable_averages.apply(tf.trainable_variables())

# 计算使用了滑动平均参数后的前向传播结果
average_y = inference(x, variable_averages, weight1, biases1, weight2, biases2)

# 计算交叉熵和平均值
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_, 1), logits=y)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
cross_entropy_mean = tf.reduce_mean(cross_entropy)

# 计算正则化损失函数
regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
# 计算模型正则化损失
regularization = regularizer(weight1) + regularizer(weight2)

# 总损失
loss = cross_entropy_mean + regularization

In [12]:
# 设置指数衰减的学习率
learn_rate = tf.train.exponential_decay(
LEARNING_RATE_BASE, global_step, mnist.train.num_examples, LEARNING_RATE_DECAY)

# 优化损失函数
train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_step=global_step)

# 通过反向传播更新神经网络中的参数
# train_op = tf.group(train_step, variable_averages_op)
with tf.control_dependencies([train_step, variable_averages_op]):
    train_op = tf.no_op(name="train")
    
# 检验滑动平均前向传播结果
correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))

# 计算模型准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    validate_feed = {
        x: mnist.validation.images,
        y_: mnist.validation.labels,
    }
    
    test_feed = {
        x: mnist.test.images,
        y_: mnist.test.labels,
    }
    
    acc = 0
    for i in range(TRAINING_STEPS):
        if i % 1000 == 0:
            validate_acc = sess.run(accuracy, feed_dict=validate_feed)
            print(i, validate_acc)
            
            if validate_acc < acc:
                break
            else:
                acc = validate_acc   
        
        xs, ys = mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_op, feed_dict={x: xs, y_: ys})
    
    test_acc = sess.run(accuracy, feed_dict=test_feed)
    print(test_acc)

0 0.0382
1000 0.9772
2000 0.9812
3000 0.9824
4000 0.9828
5000 0.9838
6000 0.9852
7000 0.9848
0.983
